Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 38.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=4510cc37938a61e5d5a166e0ab09ae533d39c9e54d7972c0bf29c7709a002eb6
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Iniciar un contexto de Spark en local

In [18]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


In [5]:
A = sc.parallelize(range(3))
A

PythonRDD[1] at RDD at PythonRDD.scala:53

Obtener la lista del RDD

In [7]:
L = A.collect()
print(type(L))
print(L)

<class 'list'>
[0, 1, 2]


Funcion de reduce con suma

In [8]:
A.reduce(lambda x,y:x+y)

3

Operación redice que busca la cadena más corta en el RDD de cadenas.

In [9]:
words = ['this', 'is', 'the', 'best','mac','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

Ejemplo función reduce mal enunciada, según como se paralelize puede causar problemas.

In [10]:
B = sc.parallelize([1,3,5,2])
B.reduce(lambda x,y: x-y)

-9

In [11]:
def largerThan(x,y):
  if len(x)>len(y): return x
  elif len(y)>len(x): return y
  else: # lengths are equals, compare lexicographically
    if x>y:
      return x
    else:
      return y

wordRDD.reduce(largerThan)

'this'

In [15]:
sc.stop()

Prueba de cuantos workers son necesarios para una tarea

no mas de 1 worker por core

In [16]:
from time import time
from pyspark import SparkContext

for j in range(1,10):
  sc = SparkContext(master="local[%d]"%(j))
  t0=time()
  for i in range(10):
    sc.parallelize([1,2]*1000000).reduce(lambda x,y:x+y)
  print("%2d executors, time=%4.3f"%(j,time()-t0))
  sc.stop()

 1 executors, time=11.620
 2 executors, time=11.189
 3 executors, time=11.245
 4 executors, time=11.298
 5 executors, time=11.272
 6 executors, time=11.426
 7 executors, time=11.662
 8 executors, time=12.132
 9 executors, time=11.712


Calcular la suma de cuadrados
con lazy evaluation

In [19]:
%%time
RDD=sc.parallelize(range(1000000))

CPU times: user 2.63 ms, sys: 0 ns, total: 2.63 ms
Wall time: 5.82 ms


define a computation

The role of the function taketime is to consume CPU cycles

In [20]:
from math import cos
def taketime(i):
  [cos(j) for j in range(100)]
  return(cos(i))

In [21]:
%%time
taketime(1)

CPU times: user 685 µs, sys: 50 µs, total: 735 µs
Wall time: 4.85 ms


0.5403023058681398

In [22]:
%%time
Internm=RDD.map(lambda x: taketime(x))

CPU times: user 62 µs, sys: 5 µs, total: 67 µs
Wall time: 73 µs


In [23]:
%%time
print('out=', Internm.reduce(lambda x, y:x+y))

out= -0.2887054679684464
CPU times: user 85.7 ms, sys: 9.68 ms, total: 95.3 ms
Wall time: 13.3 s


Se deja el cache de la primera operación para que la segunda sea más rapida

In [24]:
%%time
Intern =RDD.map(lambda x:taketime(x)).cache()

CPU times: user 5.18 ms, sys: 3.04 ms, total: 8.22 ms
Wall time: 26.8 ms


In [25]:
print(Intern.toDebugString().decode)

<built-in method decode of bytes object at 0x7f73aeb912d0>


In [26]:
%%time
print('out=', Intern.reduce(lambda x,y:x+y))

out= -0.2887054679684464
CPU times: user 87.1 ms, sys: 6.62 ms, total: 93.7 ms
Wall time: 13.4 s


In [27]:
%%time
print('out=', Intern.filter(lambda x:x>0).count())

out= 500000
CPU times: user 15.6 ms, sys: 2.97 ms, total: 18.6 ms
Wall time: 594 ms
